## My work for predicting housing prices TRAZZ PEPPER

This is my work for using a kaggle dataset and beging to try and understand the relationships between the feature varaibles and the final price of the house. I have gone through and commeted extensivly to try and explain what I am doing and why I am doing what I am doing.

In [55]:
# The main packages I used are pandas, numpy and sklearn.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [56]:
# The data was originally in a csv (train.csv)
path = "train.csv"
df = pd.read_csv(path)
df.columns.to_list()

['Id',
 'MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF'

In [57]:
# This is the function for data conditioning, this allows me formate the data for both training and testing the same.
# The first part turns categorical variables into dummies. (boolean values)


def data_conditioning(dataframe):
    dataframe.set_index("Id", inplace=True)
    categorical_columns = dataframe.select_dtypes(include=["object"]).columns

    for col in categorical_columns:
        dataframe = dataframe.drop(
            [col],
            axis=1,
        ).join(pd.get_dummies(df[col], prefix=f"{col}"))
        # Fills missing data with median (better ways to do this on improvement)
    dataframe.fillna(dataframe.median(), inplace=True)

    # Feature Engineering stuff I am trying, they make new variables from certian features
    # Turns Overall Quality into boolean as Overall quality was a numerical categorical variable.
    dataframe["Qual_Very_Excellent"] = (dataframe["OverallQual"] == 10).astype(int)
    dataframe["Qual_Excellent"] = (dataframe["OverallQual"] == 9).astype(int)
    dataframe["Qual_Very_Good"] = (dataframe["OverallQual"] == 8).astype(int)
    dataframe["Qual_Good"] = (dataframe["OverallQual"] == 7).astype(int)
    dataframe["Qual_Above_Average"] = (dataframe["OverallQual"] == 6).astype(int)
    dataframe["Qual_Average"] = (dataframe["OverallQual"] == 5).astype(int)
    dataframe["Qual_Below_Average"] = (dataframe["OverallQual"] == 4).astype(int)
    dataframe["Qual_Fair"] = (dataframe["OverallQual"] == 3).astype(int)
    dataframe["Qual_Poor"] = (dataframe["OverallQual"] == 2).astype(int)
    dataframe["Qual_Very_Poor"] = (dataframe["OverallQual"] == 1).astype(int)
    # dataframe = dataframe.drop(["OverallQual"], axis=1)

    # Turns Overall Condition into boolean as Overall condition was a numerical categorical variable.
    dataframe["Cond_Very_Excellent"] = (dataframe["OverallCond"] == 10).astype(int)
    dataframe["Cond_Excellent"] = (dataframe["OverallCond"] == 9).astype(int)
    dataframe["Cond_Very_Good"] = (dataframe["OverallCond"] == 8).astype(int)
    dataframe["Cond_Good"] = (dataframe["OverallCond"] == 7).astype(int)
    dataframe["Cond_Above_Average"] = (dataframe["OverallCond"] == 6).astype(int)
    dataframe["Cond_Average"] = (dataframe["OverallCond"] == 5).astype(int)
    dataframe["Cond_Below_Average"] = (dataframe["OverallCond"] == 4).astype(int)
    dataframe["Cond_Fair"] = (dataframe["OverallCond"] == 3).astype(int)
    dataframe["Cond_Poor"] = (dataframe["OverallCond"] == 2).astype(int)
    dataframe["Cond_Very_Poor"] = (dataframe["OverallCond"] == 1).astype(int)
    dataframe = dataframe.drop(["OverallCond"], axis=1)

    # Turns MSSubClass into boolean as MSSubClass was a numerical categorical variable.
    dataframe["1-STORY 1946 & NEWER ALL STYLES"] = (
        dataframe["MSSubClass"] == 20
    ).astype(int)
    dataframe["1-STORY 1945 & OLDER"] = (dataframe["MSSubClass"] == 30).astype(int)
    dataframe["1-STORY W/FINISHED ATTIC ALL AGES"] = (
        dataframe["MSSubClass"] == 40
    ).astype(int)
    dataframe["1-1/2 STORY - UNFINISHED ALL AGES"] = (
        dataframe["MSSubClass"] == 45
    ).astype(int)
    dataframe["1-1/2 STORY FINISHED ALL AGES"] = (dataframe["MSSubClass"] == 50).astype(
        int
    )
    dataframe["2-STORY 1946 & NEWER"] = (dataframe["MSSubClass"] == 60).astype(int)
    dataframe["2-STORY 1945 & OLDER"] = (dataframe["MSSubClass"] == 70).astype(int)
    dataframe["2-1/2 STORY ALL AGES"] = (dataframe["MSSubClass"] == 75).astype(int)
    dataframe["SPLIT OR MULTI-LEVEL"] = (dataframe["MSSubClass"] == 80).astype(int)
    dataframe["SPLIT FOYER"] = (dataframe["MSSubClass"] == 85).astype(int)
    dataframe["DUPLEX - ALL STYLES AND AGES"] = (dataframe["MSSubClass"] == 90).astype(
        int
    )
    dataframe["1-STORY PUD (Planned Unit Development) - 1946"] = (
        dataframe["MSSubClass"] == 120
    ).astype(int)
    dataframe["1-1/2 STORY PUD - ALL AGES"] = (dataframe["MSSubClass"] == 150).astype(
        int
    )
    dataframe["2-STORY PUD - 1946 & NEWER"] = (dataframe["MSSubClass"] == 160).astype(
        int
    )
    dataframe["PUD - MULTILEVEL - INCL SPLIT LEV/FOYER"] = (
        dataframe["MSSubClass"] == 180
    ).astype(int)
    dataframe["2 FAMILY CONVERSION - ALL STYLES AND AGES"] = (
        dataframe["MSSubClass"] == 190
    ).astype(int)
    dataframe = dataframe.drop(["MSSubClass"], axis=1)

    # Making some new features too see if they better correlated with the dataframe.
    # Square some of the values and see the results

    dataframe["TotalBsmtSF_squared_FE"] = dataframe["TotalBsmtSF"] ** 2
    dataframe["LotFrontage_squared_FE"] = dataframe["LotFrontage"] ** 2

    # Lots of different variables
    dataframe["TotalBsmtSF_GarageArea_FE"] = dataframe["TotalBsmtSF"] * df["GarageArea"]
    dataframe["YearBuilt_age_FE"] = 20234 - dataframe["YearBuilt"]
    dataframe["Total_Baths_FE"] = (
        dataframe["FullBath"] + (dataframe["HalfBath"] / 2) + dataframe["BsmtFullBath"]
    )
    dataframe["GarageArea_to_TotalArea_FE"] = dataframe["GarageArea"] / (
        dataframe["GrLivArea"] + dataframe["TotalBsmtSF"] + 1e-5
    )
    dataframe["GrLivArea_to_GarageArea_FE"] = dataframe["GrLivArea"] / (
        dataframe["GarageArea"] + 1e-5
    )
    dataframe["TotalBsmtSF_to_GarageArea_FE"] = dataframe["TotalBsmtSF"] / (
        dataframe["GarageArea"] + 1e-5
    )
    dataframe["Years_Since_Renovation_FE"] = 2024 - dataframe["YearRemodAdd"]
    dataframe["TotalRooms_FE"] = dataframe["TotRmsAbvGrd"] + (
        dataframe["BsmtFinType1_GLQ"] > 0
    ).astype(int)
    dataframe["LotArea_to_GarageCars_FE"] = dataframe["LotArea"] / (
        dataframe["GarageCars"] + 1e-5
    )
    dataframe["Total_SF_FE"] = (
        dataframe["GrLivArea"]
        + dataframe["TotalBsmtSF"]
        + dataframe["1stFlrSF"]
        + dataframe["2ndFlrSF"]
    )

    dataframe["Total_Bedrooms_FE"] = (
        dataframe["BedroomAbvGr"]
        + dataframe["BsmtFinType1_BLQ"]
        + (dataframe["BsmtFinType1_GLQ"] > 0).astype(int)
    )
    dataframe["Garage_SF_to_Total_SF_FE"] = dataframe["GarageArea"] / (
        dataframe["Total_SF_FE"] + 1e-5
    )
    dataframe["GrLivArea_to_LotArea_FE"] = dataframe["GrLivArea"] / (
        dataframe["LotArea"] + 1e-5
    )
    dataframe["GarageCars_to_TotalRooms_FE"] = dataframe["GarageCars"] / (
        dataframe["TotalRooms_FE"] + 1e-5
    )
    dataframe["Outdoor_Space_FE"] = (
        dataframe["WoodDeckSF"]
        + dataframe["OpenPorchSF"]
        + dataframe["EnclosedPorch"]
        + dataframe["ScreenPorch"]
    )

    return dataframe

In [58]:
# Data contitions and fills with median for missing values, and then scales the data
df = data_conditioning(dataframe=df)


# These columns are not in the test dataset so drop them
columns_to_drop = [
    "Condition2_RRAe",
    "Condition2_RRAn",
    "Condition2_RRNn",
    "Electrical_Mix",
    "Exterior1st_ImStucc",
    "Exterior1st_Stone",
    "Exterior2nd_Other",
    "GarageQual_Ex",
    "Heating_Floor",
    "Heating_OthW",
    "HouseStyle_2.5Fin",
    "MiscFeature_TenC",
    "PoolQC_Fa",
    "RoofMatl_ClyTile",
    "RoofMatl_Membran",
    "RoofMatl_Metal",
    "RoofMatl_Roll",
    "Utilities_NoSeWa",
]

df = df.drop(columns=columns_to_drop)


# Separate features and target variable
X = df.drop(columns=["SalePrice"], axis=1)
y = df["SalePrice"]

In [59]:
# Correlation Matrix to choose which variables to focus on. (however, correlation matrix too big so output a file better)
correlation_matrix = df.corr()
saleprice_correlation = (
    correlation_matrix["SalePrice"].abs().sort_values(ascending=False)
)

# Keep the variabels with correlation of 0.3 or higher (gives enough variabels but not too many to avoid overfitting)
high_corr_vars_list = saleprice_correlation[saleprice_correlation > 0.3].index.tolist()

# Shortens the amount of variables to the ones with highest correlation.
high_corr_vars = df[high_corr_vars_list]
del high_corr_vars_list[0]
X = high_corr_vars.drop(columns=["SalePrice"], axis=1)
y = high_corr_vars["SalePrice"]

In [60]:
# Split the data into training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


# Scales the data all the same
scaler = StandardScaler()
X_train_S = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_S = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# Basic Linear Regression Model to see how good we start and how to imporve.
reg = LinearRegression()


reg.fit(X_train_S, y_train)

print(
    f"The linear model fits to training data: {round(reg.score(X_train_S, y_train),3)}"
)
print(f"The linear model fits to testing data: {round(reg.score(X_test_S, y_test),3)}")

The linear model fits to training data: 0.866
The linear model fits to testing data: 0.886


In [61]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# Initialize a RandomForestRegressor with a fixed random seed for reproducibility
forest = RandomForestRegressor(random_state=1)

# Fit the model to the training data
forest.fit(X_train_S, y_train)

# Perform 10-fold cross-validation on the training data to assess model performance
cv_scores = cross_val_score(forest, X_train_S, y_train, cv=10)

# Calculate the mean and standard deviation of the cross-validation scores
mean_cv_score = np.mean(cv_scores)
std_cv_score = np.std(cv_scores)

# Print the mean cross-validation score with its standard deviation
print(f"Mean CV Score: {mean_cv_score:.4f} ± {std_cv_score:.4f}")

# Evaluate the model on the test data to check its performance on unseen data
test_score = forest.score(X_test_S, y_test)

# Print the test score (R² coefficient of determination)
print(f"Test Score: {test_score:.4f}")

Mean CV Score: 0.8433 ± 0.0603
Test Score: 0.9013


In [62]:
from sklearn.model_selection import GridSearchCV

# This sets up a grid search to see if I can optimize the forest regression further.
param_grid = {
    "n_estimators": [900, 1000, 1100, 2000],
    "max_depth": [None, 10, 20, 30],
    "max_features": ["log2"],
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=forest,
    param_grid=param_grid,
    scoring="r2",
    cv=5,
    n_jobs=-1,
)

# Fit GridSearchCV
grid_search.fit(X_train_S, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best CV Score: {best_score}")

# Evaluate on the test set
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test_S, y_test)
print(f"Test Score: {test_score}")

Best Parameters: {'max_depth': 20, 'max_features': 'log2', 'n_estimators': 900}
Best CV Score: 0.8467276655692972
Test Score: 0.883593947094109


In [65]:
# Here I read in the test file, condition the data and then predict the outomes to a csv file.
path = "test.csv"
df = pd.read_csv(path)
df_index = df.Id

df_test = data_conditioning(dataframe=df)

df_test_S = pd.DataFrame(scaler.transform(df_test), columns=df_test.columns)
df_test_S = df_test_S[high_corr_vars_list]

y_prediction = forest.predict(df_test_S)

output_df = pd.DataFrame({"Id": df_index, "SalePrice": y_prediction})
output_df.set_index("Id").to_csv("Housing_Price_Estimates_TP.csv")

In [66]:
# prints the R^2 value to see how the model looks to be doing.
y_pred = forest.predict(X_test_S)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(rmse)

26530.89142725713


## Ideas moving forward
- I seem to have too many varaibels, but as I increase the number of variables the accuracy goes up so im not sure?
- Potential for multicolinearity (too many variables look to reduce some)
- Have missing values, better way to estimate the values of those. (instead of median)